In [9]:
import pandas as pd # Importa la biblioteca pandas.

# Cargar datos de prueba desde un archivo zip.
test_data = pd.read_csv(
    "../files/input/test_data.csv.zip", # Especifica la ruta relativa al archivo .zip que contiene los datos de prueba.
    index_col=False, # Indica que no se debe usar ninguna columna como índice del DataFrame.    
    compression="zip", # Especifica que el archivo está comprimido como un .zip.
)

# Cargar datos de entrenamiento desde un archivo zip.
train_data = pd.read_csv(
    "../files/input/train_data.csv.zip", # Especifica la ruta relativa al archivo .zip que contiene los datos de entrenamiento.
    index_col=False, # Indica que no se debe usar ninguna columna como índice del DataFrame.
    compression="zip", # Especifica que el archivo está comprimido como un .zip.
)

In [10]:
#
# limpieza de datos
#

# Cree la columna 'Age' a partir de la columna 'Year' (Asuma que el año actual es 2021).
test_data['Age'] = 2021 - test_data['Year']
train_data['Age'] = 2021 - train_data['Year']


# Elimine las columnas 'Year' y 'Car_Name'.
test_data = test_data.drop(columns=['Year','Car_Name'])
train_data = train_data.drop(columns=['Year','Car_Name'])

In [11]:
#
# separacion de los datos
#

# Elimina la columna 'Present_Price' del DataFrame train_data para crear el conjunto de características de entrenamiento.
x_train = train_data.drop(columns="Present_Price")

# Extrae la columna 'Present_Price' de train_data y la usa como el conjunto de etiquetas de entrenamiento.
y_train = train_data["Present_Price"]

# Elimina la columna 'Present_Price' del DataFrame test_data para crear el conjunto de características de prueba
x_test = test_data.drop(columns="Present_Price")

# Extrae la columna 'Present_Price' de test_data y la usa como el conjunto de etiquetas de prueba.
y_test = test_data["Present_Price"]

In [12]:
#
# eleccion modelo y transformaciones
#

from sklearn.compose import ColumnTransformer # Aplica transformaciones específicas a subconjuntos de columnas (categóricas y numéricas).
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline # Permite encadenar varios pasos como preprocesamiento, selección de características y modelado.
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler # Herramientas para escalar y codificar datos.
from sklearn.feature_selection import f_regression,SelectKBest # Selecciona las mejores características basadas en una métrica estadística.

# Definición de las columnas categóricas y numéricas.
categorical_features=['Fuel_Type','Selling_type','Transmission'] # Lista de columnas categóricas.
numerical_features= x_train.columns.difference(categorical_features + ["default"]).tolist()  # Lista de columnas numéricas.

# Configuración del preprocesador.
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features), # Codifica variables categóricas.
        ('scaler',MinMaxScaler(),numerical_features), # Escala las variables numéricas para tener media 0 y desviación estándar 1.
    ],
    remainder='passthrough'  # Pasa sin cambios las columnas que no están incluidas en 'transformers'.
)

# Creación del pipeline.
pipeline=Pipeline(
    [
        ("preprocessor",preprocessor), # Aplica las transformaciones del preprocesador
        ('feature_selection',SelectKBest(f_regression)), # Selecciona las características más relevantes.
        ('classifier', LinearRegression()) # Aplica el modelo de regresión lineal como clasificador.
    ]
)

In [13]:
#
# hiperparametros y ajuste
#

from sklearn.model_selection import GridSearchCV  # Herramienta para buscar automáticamente la mejor combinación de hiperparámetros.

# Definición de la rejilla de parámetros para la búsqueda.
param_grid = {
    'feature_selection__k': range(1, 15),  # Número de características a seleccionar en SelectKBest (de 1 a 14).
    'classifier__fit_intercept': [True, False],  # Si se ajusta el intercepto (constante) en el modelo de clasificación.
    'classifier__positive': [True, False]  # Si se restringen los coeficientes a ser positivos (solo válido para ciertos clasificadores).
}

# Configuración de GridSearchCV.
model = GridSearchCV(
    pipeline,  # Pipeline que incluye preprocesamiento, selección de características y modelo de clasificación.
    param_grid,  # Rejilla de hiperparámetros definida anteriormente.
    cv=10,  # Número de particiones para la validación cruzada (10 particiones en este caso).
    scoring="neg_mean_absolute_error",  # Métrica de evaluación: error absoluto medio negativo (se usará su valor positivo como resultado).
    n_jobs=-1,  # Usa todos los núcleos disponibles para procesar en paralelo, acelerando la búsqueda.
)
# Ajusta el modelo en el conjunto de entrenamiento.
model.fit(x_train, y_train)

# Imprimir los mejores parámetros y el mejor score.
print("Mejores parámetros:", model.best_params_)
print("Mejor precisión balanceada:", model.best_score_)

Mejores parámetros: {'classifier__fit_intercept': True, 'classifier__positive': True, 'feature_selection__k': 11}
Mejor precisión balanceada: -1.765768769272933


Mejores parámetros: {'classifier__fit_intercept': True, 'classifier__positive': True, 'feature_selection__k': 11}

Mejor precisión balanceada: -1.765768769272933

In [14]:
#
# guardar modelo
#

import gzip  # Para comprimir y descomprimir archivos.
import pickle  # Para serializar y deserializar objetos de Python, como modelos de machine learning.
import os  # Para manejo de directorios.

models_dir = '../files/models'  # Define la ruta donde se guardarán los modelos.
os.makedirs(models_dir, exist_ok=True)  # Crea el directorio si no existe.

model_path = "../files/models/model.pkl.gz"  # Define la ruta completa para el archivo del modelo.

with gzip.open(model_path, "wb") as f:  # Abre el archivo en modo escritura binaria ('wb') y comprime el contenido.
    pickle.dump(model, f)  # Serializa el objeto del modelo y lo guarda en el archivo comprimido.

model_path  # Devuelve la ruta donde se guardó el modelo.

'../files/models/model.pkl.gz'

In [15]:
#
# funcion ejecutura y salva las metricas
#

import json  # Biblioteca para manejar datos en formato JSON.
import os  # Biblioteca para operaciones de manejo de directorios y archivos.
from sklearn.metrics import r2_score, mean_squared_error, median_absolute_error  # Métricas de evaluación para regresión.

def calculate_and_save_metrics(model, X_train, X_test, y_train, y_test):
    """
    Calcula métricas de evaluación para un modelo entrenado en conjuntos de entrenamiento y prueba,
    e imprime y guarda estas métricas en un archivo JSON.
    """
    # Hacer predicciones en los conjuntos de entrenamiento y prueba.
    y_train_pred = model.predict(X_train)  # Predicciones para el conjunto de entrenamiento.
    y_test_pred = model.predict(X_test)  # Predicciones para el conjunto de prueba.

    # Calcular métricas para el conjunto de entrenamiento.
    metrics_train = {
        'type': 'metrics',  # Identifica que es un conjunto de métricas.
        'dataset': 'train',  # Indica que estas métricas son del conjunto de entrenamiento.
        'r2': float(r2_score(y_train, y_train_pred)),  # Coeficiente de determinación R².
        'mse': float(mean_squared_error(y_train, y_train_pred)),  # Error cuadrático medio (MSE).
        'mad': float(median_absolute_error(y_train, y_train_pred))  # Error absoluto mediano (MAD).
    }

    # Imprimir métricas del conjunto de entrenamiento.
    print("Train Metrics:", metrics_train)

    # Calcular métricas para el conjunto de prueba.
    metrics_test = {
        'type': 'metrics',  # Identifica que es un conjunto de métricas.
        'dataset': 'test',  # Indica que estas métricas son del conjunto de prueba.
        'r2': float(r2_score(y_test, y_test_pred)),  # Coeficiente de determinación R².
        'mse': float(mean_squared_error(y_test, y_test_pred)),  # Error cuadrático medio (MSE).
        'mad': float(median_absolute_error(y_test, y_test_pred))  # Error absoluto mediano (MAD).
    }

    # Imprimir métricas del conjunto de prueba.
    print("Test Metrics:", metrics_test)

    # Crear carpeta de salida si no existe.
    output_dir = '../files/output'  # Directorio donde se guardarán las métricas.
    os.makedirs(output_dir, exist_ok=True)  # Crea el directorio si no existe.

    # Guardar las métricas en un archivo JSON.
    output_path = os.path.join(output_dir, 'metrics.json')  # Ruta completa del archivo JSON.
    with open(output_path, 'w') as f:  # Abrir el archivo en modo escritura ('w') para crear un archivo limpio.
        f.write(json.dumps(metrics_train) + '\n')  # Escribir las métricas de entrenamiento en formato JSON.
        f.write(json.dumps(metrics_test) + '\n')  # Escribir las métricas de prueba en formato JSON.

In [16]:
# Ejemplo de uso:
calculate_and_save_metrics(model, x_train, x_test, y_train, y_test)

Train Metrics: {'type': 'metrics', 'dataset': 'train', 'r2': 0.8916962358587399, 'mse': 5.8746462805982045, 'mad': 1.092912344019548}
Test Metrics: {'type': 'metrics', 'dataset': 'test', 'r2': 0.7325716754123306, 'mse': 32.56667275386626, 'mad': 1.5033540603205706}
